In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import pandas as pd
from qdrant_client import QdrantClient
import os
import glob
import pandas as pd
import uuid
from langchain.vectorstores import Qdrant
from qdrant_client.models import PointStruct
from dotenv import load_dotenv
load_dotenv()

True

In [22]:

# ✅ Set up Embeddings Model
model_name = "models/embedding-001"
embeddings = GoogleGenerativeAIEmbeddings(model=model_name)

# ✅ Connect to Qdrant
qdrant = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

# ✅ Recreate collection with correct vector size
qdrant.recreate_collection(
    collection_name="my_collection",
    vectors_config={"size": 768, "distance": "Cosine"}  # Ensure correct size
)

# ✅ Process multiple CSV files
all_vectors = []
csv_files = glob.glob("C:\\Users\\dhruv sharma\\OneDrive\\Desktop\\*.csv")

def getText():
    global all_vectors
    if not csv_files:
        print("❌ No CSV files found.")
        return
    
    for file in csv_files:
        df = pd.read_csv(file, encoding="ISO-8859-1")
        
        for _, row in df.iterrows():
            # ✅ Extract only non-null column values dynamically
            metadata = {col: row[col] for col in df.columns if pd.notna(row[col])}
            metadata["source_file"] = os.path.basename(file)  # Store file name

            # ✅ Combine all text fields for embedding
            text_data = " ".join(str(row[col]) for col in df.columns if pd.notna(row[col]))

            if text_data:  # Ensure valid text
                vector = embeddings.embed_query(text_data)  # Generate vector
                
                all_vectors.append(
                    PointStruct(id=str(uuid.uuid4()), vector=vector, payload=metadata)
                )

def store_embeddings():
    global all_vectors
    if not all_vectors:
        print("❌ No embeddings to store.")
        return

    qdrant.upsert(collection_name="my_collection", points=all_vectors)
    print(f"✅ Successfully stored {len(all_vectors)} embeddings in Qdrant.")

# ✅ Execute
getText()
store_embeddings()

C:\Users\dhruv sharma\AppData\Local\Temp\ipykernel_6028\395930160.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


✅ Successfully stored 48 embeddings in Qdrant.


In [20]:
from qdrant_client.models import Filter  # Import the missing Filter class

qdrant = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

qdrant.delete(
    collection_name="my_collection",
    points_selector=Filter(must=[])  # Deletes all points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)